In [ ]:
# Importação e leitura do dataset
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/01. Projetos de Ciência e Análise de Dados/Modelagem de risco de Crédito/Home Credit Default/application_train.csv')
df = df.sample(frac=1, random_state=42)

In [ ]:
# Volumetria do dataset
df.shape

(307511, 122)

In [ ]:
# Visualizando
pd.set_option('display.max_columns', None)
df.head(3)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
245895,384575,0,Cash loans,M,Y,N,2,207000.0,465457.5,52641.0,418500.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.009630,-13297,-762,-637.0,-4307,19.0,1,1,0,1,0,0,Sales staff,4.0,2,2,THURSDAY,11,0,0,0,0,1,1,Business Entity Type 3,0.675878,0.604894,0.000527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-2.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0
98194,214010,0,Cash loans,F,Y,Y,0,247500.0,1281712.5,48946.5,1179000.0,Unaccompanied,Commercial associate,Higher education,Single / not married,House / apartment,0.006852,-14778,-1141,-1610.0,-4546,11.0,1,1,0,1,0,1,Managers,1.0,3,3,THURSDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.430827,0.425351,0.712155,0.0753,0.0568,0.997,0.9592,0.1326,0.08,0.0517,0.4167,0.2917,0.0735,0.0601,0.0844,0.0058,0.1118,0.0756,0.0566,0.994,0.9216,0.0523,0.0806,0.0345,0.3333,0.0417,0.0445,0.0652,0.0857,0.0,0.0,0.076,0.0568,0.997,0.9597,0.1335,0.08,0.0517,0.4167,0.2917,0.0748,0.0611,0.0859,0.0058,0.1142,reg oper account,block of flats,0.0754,Monolithic,No,2.0,0.0,2.0,0.0,-1071.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
36463,142232,0,Cash loans,F,Y,N,0,202500.0,495000.0,39109.5,495000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-17907,-639,-2507.0,-1461,4.0,1,1,1,1,0,0,Sales staff,2.0,2,2,TUESDAY,16,0,0,0,0,0,0,Self-employed,0.527239,0.531760,0.207964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,0.0,5.0,0.0,-1435.

In [ ]:
# Balanceamento da amostra
df['TARGET'].value_counts(1)

0    0.919271
1    0.080729
Name: TARGET, dtype: float64

In [ ]:
# Separando bases

# Dados somente de NÃO-EVENTO
df_00 = df[df['TARGET'] == 0].copy()
df_00 = df_00.sample(frac=1)

# Dados somente de EVENTO
df_01 = df[df['TARGET'] == 1].copy()
df_01 = df_01.sample(frac=1)

In [ ]:
# Volumetrias
print(f'df_00: {df_00.shape}')
print(f'df_01: {df_01.shape}')
print(f'df:    {df.shape}')

df_00: (282686, 122)
df_01: (24825, 122)
df:    (307511, 122)


In [ ]:
# Ajustando a base onde só há NÃO-EVENTO (TARGET 0)

import numpy as np

safra_lista = [
    "202112", "202201", "202202", "202203",
    "202204", "202205", "202206", "202207",
    "202208", "202209", "202210", "202211", "202212" ]

# Criando a coluna 'Safra'
df_00['DATA_REF'] = ''

# Taxa de evento na base
tx = 100*df['TARGET'].value_counts(1)[1]

# Variação superior
var_sup = 0.75

# Variação inferior
var_inf = 0.75

# Proporções
import random
proporcoes = []

for i in range(0,13):
  proporcoes.append(random.uniform(tx-var_inf , tx+var_sup))

# Calculando o número de linhas para cada parte
num_linhas_total = len(df_00)
num_partes = len(proporcoes)
num_linhas_partes = np.round(np.array(proporcoes) / sum(proporcoes) * num_linhas_total).astype(int)

# Garantindo que a soma das partes seja igual ao número total de linhas
num_linhas_partes[-1] += num_linhas_total - sum(num_linhas_partes)

# Embaralhando as linhas do DataFrame
df_00 = df_00.sample(frac=1, random_state=42)

# Criando a coluna 'Safra' e atribuindo valores da lista
inicio = 0
for i, num_linhas in enumerate(num_linhas_partes):
    fim = inicio + num_linhas
    df_00.iloc[inicio:fim, df_00.columns.get_loc('DATA_REF')] = safra_lista[i]
    inicio = fim


In [ ]:
# Ajustando a base onde só há EVENTO (TARGET 1)

import numpy as np


safra_lista = [
    "202112", "202201", "202202", "202203",
    "202204", "202205", "202206", "202207",
    "202208", "202209", "202210", "202211", "202212" ]

# Criando a coluna 'Safra'
df_01['DATA_REF'] = ''

# Taxa de evento na base
tx = 100*df['TARGET'].value_counts(1)[1]

# Variação superior
var_sup = 0.75

# Variação inferior
var_inf = 0.75

# Proporções
import random
proporcoes = []

for i in range(0,13):
  proporcoes.append(random.uniform(tx-var_inf , tx+var_sup))


# Calculando o número de linhas para cada parte
num_linhas_total = len(df_01)
num_partes = len(proporcoes)
num_linhas_partes = np.round(np.array(proporcoes) / sum(proporcoes) * num_linhas_total).astype(int)

# Garantindo que a soma das partes seja igual ao número total de linhas
num_linhas_partes[-1] += num_linhas_total - sum(num_linhas_partes)

# Embaralhando as linhas do DataFrame
df_01 = df_01.sample(frac=1, random_state=42)

# Criando a coluna 'Safra' e atribuindo valores da lista
inicio = 0
for i, num_linhas in enumerate(num_linhas_partes):
    fim = inicio + num_linhas
    df_01.iloc[inicio:fim, df_01.columns.get_loc('DATA_REF')] = safra_lista[i]
    inicio = fim



In [ ]:
# Concatenando
df_agrupado = pd.concat([df_00 , df_01] , axis=0)

In [ ]:
# Verificando o balancemanto no dataframe com safra
df_agrupado['TARGET'].value_counts(1)

0    0.919271
1    0.080729
Name: TARGET, dtype: float64

In [ ]:
# Verificando as taxas de evento e volumetria em cada safra
df_agrupado.groupby('DATA_REF').agg({'TARGET': 'mean' , 'DATA_REF':'count'})

,TARGET,DATA_REF
DATA_REF,,
202112,0.067910,25622
202201,0.084157,22553
202202,0.079494,22618
202203,0.078147,22880
202204,0.080641,24330
202205,0.078107,25798
202206,0.077735,23130
202207,0.081684,22844
202208,0.086764,23316


In [ ]:
# Embaralhando o dataset
df_agrupado = df_agrupado.sample(frac=1).reset_index(drop=True)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification


# Identificar colunas categóricas e contínuas
categorical_columns = df_agrupado.select_dtypes(include=['object']).columns.tolist()
continuous_columns = df_agrupado.select_dtypes(include=['float64', 'int64']).columns.tolist()
all_columns = categorical_columns + continuous_columns

# Adicionar uma coluna combinada de todas as variáveis
df_agrupado['combined_strata'] = df_agrupado[all_columns].astype(str).agg('_'.join, axis=1)

# Dividir o DataFrame em treino e teste sem estratificação
df_train, df_sample = train_test_split(df_agrupado, test_size=0.5, random_state=42)

# Remover a coluna temporária
df_agrupado = df_agrupado.drop('combined_strata', axis=1)

In [ ]:
# Criando amostra do dataset

# Tamanho da amostra
frac = 0.5

# Cria a amostra
df_sample = df_agrupado.sample(frac=frac, random_state=42)

In [ ]:
# Salvando o dataframe completo e a amostra

# Nomes
nome_df_completo  =  'application_train_safra'
nome_amostra      =  'application_train_safra_SAMPLE'

# Path
path = '/content/drive/MyDrive/Colab Notebooks/01. Projetos de Ciência e Análise de Dados/Modelagem de risco de Crédito/Home Credit Default/'

# Salvando
# df_agrupado.to_csv(f'{path}{nome_df_completo}.csv', index=False , mode='w')
df_sample.to_csv(f'{path}{nome_amostra}.csv', index=False ,mode='w')